# Lab 2: Convolution & Discrete Fourier Transform

In [ ]:
import commonfunctions as cf # this a custom module found the commonfunctions.py
import skimage.io as io
import matplotlib.pyplot as plt
import numpy as np
from skimage.color import rgb2gray
from skimage.color import rgba2rgb
from scipy import fftpack
from scipy.signal import convolve2d
from skimage.util import random_noise
from skimage.exposure import rescale_intensity

# Show the matlpotlib figures inside the notebook
%matplotlib inline

##### Helper functions:
cf.show_images(images, titles=None)
<br>
Given a list of images and -(optionally)- a list of titles, it plots them using matplotlib with their respective titles 

### Part 1
#### Understanding the Concept of Inverse DFT

In this part, we will construct matrices in the frequency domain and see how they look in the 2D space domain

In [ ]:
# This function maps matrices from frequency to space then plots them
def plot_image_from_freq(freq_domain_mat):
    inverse_fft_mat = fftpack.ifft2(freq_domain_mat) # Inverse FFT is a fast version of inverse DFT
    # Due to approximations, the returned matrix contains complex numbers
    # So, we get the magnitude to be able to plot the image
    image = np.abs(inverse_fft_mat) 

    cf.show_images([image], titles=['Image in Space Domain'])

In [ ]:
# We will first try to construct a matrix in the frequency domain that makes a vertically moving ripple in the space domain

freq_domain_mat = np.zeros([21,21])
freq_domain_mat[9, 10] = 255 # The choice of the value '1' is arbitrary
freq_domain_mat[11, 10] = 255
plot_image_from_freq(freq_domain_mat)

In [ ]:
# TODO: construct a matrix in the frequency domain that makes a HORIZONTALLY moving ripple in the space domain

freq_domain_mat = np.zeros([21,21])
freq_domain_mat[10, 9] = 1
freq_domain_mat[10, 11] = 1
plot_image_from_freq(freq_domain_mat)

In [ ]:
# Now a ripple moving diagonally
freq_domain_mat = np.zeros([21,21])
freq_domain_mat[9, 9] = 1
freq_domain_mat[11, 11] = 1
plot_image_from_freq(freq_domain_mat)

We recommend you try to construct different matrices like the previous ones and see if their space domain representation is what you expect

### Part 2
#### Understanding Image Filtering in the Frequency Domain

In this part, we will try to alter an image by multiplying it by a filter in the frequency domain.

In [ ]:
# This function applies a filter to an image in the frequency domain
# and plots multiple images describing the process
def apply_filter_in_freq(img, f):
    img_in_freq = fftpack.fft2(img)
    
    # we supply the img shape here to make both the filter and img have the same shape to be able to multiply
    filter_in_freq = fftpack.fft2(f, img.shape)
    filtered_img_in_freq = np.multiply(img_in_freq, filter_in_freq)
    filtered_img = fftpack.ifft2(filtered_img_in_freq)
    temp = (abs(filtered_img.flatten()))
    
    cf.show_images([img,
                fftpack.fftshift(np.log(np.abs(img_in_freq)+1)), # log for better intensity scale, 
                                                                 # shift to make zero freq at center
                fftpack.fftshift(np.log(np.abs(filter_in_freq)+1)),
                fftpack.fftshift(np.log(np.abs(filtered_img_in_freq)+1)),
                np.abs(filtered_img)
                ], ['Image', 'Image in Freq. Domain', 'Filter in Freq. Domain', 'Filtered Image in Freq. Domain', 'Filtered Image'])

In [ ]:
for image in ["imgs/bird.jpg", "imgs/circuit.tif", "imgs/Picture1.png", "imgs/Picture2.png"]:
   
    # Let's try some filters on a sample image
    img = io.imread(image)
    
    if img.ndim == 3 and (img.shape)[2] == 3:
        img = rgb2gray(img)
    elif img.ndim == 3 and (img.shape)[2] == 4:
        img = rgba2rgb(img)
        img = rgb2gray(img)
    else:
        img = rescale_intensity(img, in_range=(0, 255), out_range=(0, 1.0))
        

    # This is a low pass filter (more on that in the upcoming lectures)
    f=np.array([
        [1,2,1],
        [2,4,2],
        [1,2,1]
    ])

    apply_filter_in_freq(img, f)

In [ ]:

# This is a high pass filter (more on that in the upcoming lectures)
f=np.array([
    [ 0,-1, 0],
    [-1, 4,-1],
    [ 0,-1, 0]
])

for image in ["imgs/bird.jpg", "imgs/circuit.tif", "imgs/Picture1.png", "imgs/Picture2.png"]:
   
    # Let's try some filters on a sample image
    img = io.imread(image)
    
    if img.ndim == 3 and (img.shape)[2] == 3:
        img = rgb2gray(img)
    elif img.ndim == 3 and (img.shape)[2] == 4:
        img = rgba2rgb(img)
        img = rgb2gray(img)
    else:
        img = rescale_intensity(img, in_range=(0, 255), out_range=(0, 1.0))
        
    apply_filter_in_freq(img, f)

#### TODO
What happened to the filtered images in the two previous examples and why?
Low pass filter: made the image blured. High pass filter: show the edges because the edges are in high frequency.

### Part 3
#### Understanding Image Filtering in the Space Domain Through Convolution 

In this part, you are required to convolve a couple of filters on imgs/bird.jpg which are shown in the next figure.

<img src='imgs/filters.png'></img>

#### Functions you might need:
-> convolve2d(img, f), for documentation: <br>

Visit this link (https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.convolve2d.html)
<br>
Or
<br>
Press shift+tab after writing 'convolve2d' in a code cell

-> random_noise(img, mode) (https://scikit-image.org/docs/dev/api/skimage.util.html#skimage.util.random_noise)

In [ ]:
# TODO: Construct the required filters (hint: you can do it as we did in the previous part)
f1 = np.ones((3,3)) * (1/9)

f2 = np.array([
    [ 1,1, 1],
    [1, -8,1],
    [ 1,1, 1]
])


f3 = np.array([
    [1, 1, 1],
    [0, 0, 0],
    [-1, -1, -1]
])

f4 = np.array([
    [1, 0, -1],
    [1, 0, -1],
    [1, 0, -1]
])
    
for image in ["imgs/bird.jpg", "imgs/circuit.tif", "imgs/Picture1.png", "imgs/Picture2.png"]:

    # TODO: Read an image and convert it to grayscale
    img = io.imread(image)
    
    if img.ndim == 3 and (img.shape)[2] == 3:
        img = rgb2gray(img)
    elif img.ndim == 3 and (img.shape)[2] == 4:
        img = rgba2rgb(img)
        img = rgb2gray(img)
    else:
        img = rescale_intensity(img, in_range=(0, 255), out_range=(0, 1.0))


    # TODO: Apply noise to the image and save it in another variable
    noisy_img = random_noise(img, mode="s&p")


    # TODO: Convolve the noisy image with f1 and the rest of the filters with the original image
    img_f1 = convolve2d(noisy_img, f1, boundary="symm")

    img_f2 = convolve2d(img, f2, boundary="symm")

    img_f3 = convolve2d(img, f3, boundary="symm")

    img_f4 = convolve2d(img, f4, boundary="symm")
    
    
    # Show the images
    cf.show_images([rescale_intensity(x, in_range=(0.0,1.0), out_range=(0, 255)) for x in [img,noisy_img, img_f1,img_f2,img_f3,img_f4]],['Original','Noisy Image', 'Filtered 1','Filtered 2','Filtered 3','Filtered 4'])

What are the uses of these filters ?
First one: averaging
second one: edge detection.
third: detecting horizontal edges.
fourth: detecting vertical edges

For the second filter, how can we make the output more descriptive? with increasing the filter size 

In [ ]:
image = np.array([
    [5000, 0, 0],
    [2500, 3500, 1500],
    [0, 0, 0]
])

cf.show_images([image], ["image"])